In [ ]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
file_path = "/Users/pdmcguckian/Documents/IoT/Project/SIoT/Data & Modelling/BLEData.csv"
data_set = pd.read_csv(file_path)
dataset = pd.DataFrame(data_set)
dataset.fillna(0)

print("Dimension of dataset: ", dataset.shape)
tmp_str = "Feature(attribute)     DataType"; print(tmp_str+"\n"+"-"*len(tmp_str))
print(dataset.dtypes)

In [ ]:
dataset = dataset.drop(['_id'], axis=1)

for i in range(len(dataset['timeStampID'])):
    try:
        unix = dataset['timeStampID'][i]
        timestr = time.ctime(unix)[11:19]
        dataset['timeStampID'][i] = timestr
    except:
        dataset['timeStampID'][i] = 0

dataset.head()

In [ ]:
times = [[]]
bdrms = [[]]
ktchns = [[]]
lvngrms = [[]]

day = 0
for index, row in dataset.iterrows():
    if row['timeStampID'] == "12:00:00":
        day += 1
        print(row)
        times.append([])
        bdrms.append([])
        ktchns.append([])
        lvngrms.append([])

    times[day].append(row['timeStampID'])
    bdrms[day].append(row['rssiBDRM'])
    ktchns[day].append(row['rssiKTCHN'])
    lvngrms[day].append(row['rssiLVNGRM'])

In [ ]:
for i in range(1, len(times)-1):
    fig, ax = plt.subplots()
    ax.plot(times[i][:1400], bdrms[i][:1400], linewidth=1.0, label = "Bedroom")
    ax.plot(times[i][:1400], lvngrms[i][:1400], linewidth=1.0, label = "Living Room")
    ax.plot(times[i][:1400], ktchns[i][:1400], linewidth=1.0, label = "Kitchen")
    
    every_nth = 240
    for n, label in enumerate(ax.xaxis.get_ticklabels()):
        if n % every_nth != 0:
            label.set_visible(False)

    plt.show()

In [ ]:
bdrmsbools = [[]]
ktchnsbools = [[]]
lvngrmsbools = [[]]
wybool = [[]]

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)


#for i in range(len(times)):
#        bdrms[i] = running_mean(bdrms[i], 5)
#        lvngrms[i] = running_mean(lvngrms[i], 5)
#        ktchns[i] = running_mean(ktchns[i], 5)

day = 0
for i in range(len(lvngrms)):
    for j in range(len(lvngrms[i])):
        bdrm = bdrms[i][j]
        lvngrm = lvngrms[i][j]
        ktchn = ktchns[i][j]

        if bdrm == -100 and ktchn == -100 and lvngrm == -100:
            wybool[day].append(1)
            ktchnsbools[day].append(0)
            bdrmsbools[day].append(0)
            lvngrmsbools[day].append(0)

        elif bdrm >= ktchn and bdrm >= lvngrm:
            wybool[day].append(0)
            ktchnsbools[day].append(0)
            bdrmsbools[day].append(1)
            lvngrmsbools[day].append(0)

        elif ktchn >= bdrm and ktchn >= lvngrm:
            wybool[day].append(0)
            ktchnsbools[day].append(1)
            bdrmsbools[day].append(0)
            lvngrmsbools[day].append(0)

        elif lvngrm >= bdrm and lvngrm >= ktchn:
            wybool[day].append(0)
            ktchnsbools[day].append(0)
            bdrmsbools[day].append(0)
            lvngrmsbools[day].append(1)

        if sum(wybool[-1]) == 1440:
            print()
            wybool.pop()
            ktchnsbools.pop()
            bdrmsbools.pop()
            lvngrmsbools.pop()
            day-=1
    day+=1
    wybool.append([])
    ktchnsbools.append([])
    bdrmsbools.append([])
    lvngrmsbools.append([])


In [ ]:

for i in range(1, len(wybool)):
    try:
        fig, ax = plt.subplots()
        ax.plot(times[i][:1440], bdrmsbools[i][:1440], linewidth=1.0, label = "Bedroom")
        ax.plot(times[i][:1440], lvngrmsbools[i][:1440], linewidth=1.0, label = "Living Room")
        ax.plot(times[i][:1440], ktchnsbools[i][:1440], linewidth=1.0, label = "Kitchen")
        
        every_nth = 240
        for n, label in enumerate(ax.xaxis.get_ticklabels()):
            if n % every_nth != 0:
                label.set_visible(False)

        plt.show()
    except:
        pass